In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
# Chọn file kaggle.json tải về ở bước 2 để tải lên

!mkdir '/root/.kaggle'
!cp kaggle.json '/root/.kaggle'
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets list
# Chạy lệnh liệt kê các bộ dữ liệu của bạn để đảm bảo bạn đã cài đặt thành công

Saving kaggle.json to kaggle.json
ref                                                           title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
salvatorerastelli/spotify-and-youtube                         Spotify and Youtube                                9MB  2023-03-20 15:43:25           3878        152  1.0              
ashishraut64/global-methane-emissions                         Global Emissions.                                 31KB  2023-03-27 09:02:51           1088         34  1.0              
thitiwat/top-100-most-expensive-football-transfers            ⚽ Top 100 most expensive football transfers        2KB  2023-03-27 04:06:28            650         26  1.0              
davidms8/product-sales-detail                      

In [ ]:
!kaggle datasets download -d hasibalmuzdadid/shoe-vs-sandal-vs-boot-dataset-15k-images

 94% 44.0M/47.0M [00:03<00:00, 20.2MB/s]
100% 47.0M/47.0M [00:03<00:00, 15.9MB/s]


In [ ]:
!unzip -q shoe-vs-sandal-vs-boot-dataset-15k-images.zip

In [ ]:
raw_folder = "/content/Shoe vs Sandal vs Boot Dataset/"

In [ ]:
def save_data(raw_folder=raw_folder):
    dest_size=(128,128)
    print("Start Processing")
    pixels = []
    labels = []

    #load tung thu muc con
    for folder in listdir(raw_folder):
        if folder!='.DS_Store':
            print("Folder=", folder)
            # load cac anh
            for file in listdir(raw_folder+folder):
                print("File=",file)
                pixels.append(cv2.resize(cv2.imread(raw_folder+folder+"/"+file), dsize=(dest_size)))#doc cac anh va luu lam pixels
                labels.append(folder)#lay labels la cac thu muc con chua cac anh

    pixels=np.array(pixels)
    labels = np.array(labels) #reshape(-1,1)

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)
    # print(labels)

    file = open('pix.data', 'wb')
    # ket xuat thong tin vao tep do
    pickle.dump((pixels, labels), file)
    # dong file
    file.close()
    return

def load_data():
    file = open('pix.data', 'rb')
    (pixels, labels)= pickle.load(file)
    file.close()
    # print(pixels.shape)
    # print(labels.shape)
    return pixels, labels

In [ ]:
print(pixels.shape)

In [ ]:
save_data()
X,y=load_data()
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=100)
print(X_train.shape)
print(y_train.shape)

In [ ]:
def get_model():
    model_vgg16_conv=VGG16(weights='imagenet', include_top=False)

    #dong bang cac layer
    for layer in model_vgg16_conv.layers:
        layer.trainable=False

    #tao model
    input = Input(shape=(128,128,3), name='image_input')
    output_vgg16_conv=model_vgg16_conv(input)

    #them cac layer FC va Dropout
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation='softmax    ', name='predictions')(x)

    #compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return my_model


In [ ]:
vggmodel = get_model()

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
filepath = "weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#xay dung dao tao hinh anh de tang cuong du lieu
aug = ImageDataGenerator(
                         rotation_range=20,
                         zoom_range=0.1,
                         rescale=1./255,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         brightness_range=[0.2,1.5],
                         fill_mode="nearest",
                         shear_range = 1
                         )
#aug_val=ImageDataGenerator(rescale=1./255)
train_generator = aug.flow(X_train, y_train,batch_size= 32,shuffle=True, seed = 42)
n_augmented_images = train_generator.n
print(n_augmented_images)
#vggmodel.fit_generator(train_generator, epochs= 10, validation_data = aug.flow(X_test, y_test, batch_size=64), callbacks=callbacks_list)

                       

                       


12000


In [ ]:
# tạo đối tượng ImageDataGenerator cho train set
aug_train = ImageDataGenerator(
    rotation_range=20, 
    zoom_range=0.1,
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.2,1.5], 
    fill_mode="nearest"
)
batch_size = 64
# Tạo generator cho train set
train_generator = aug_train.flow(X_train, y_train, batch_size, shuffle=True)

# Lấy số lượng ảnh được sinh ra trong quá trình augmenting
n_augmented_images = train_generator.n

# Tính số lượng bước để huấn luyện cho mỗi epoch
steps_per_epoch = int(np.ceil(n_augmented_images / batch_size))

# Tạo một mảng numpy để lưu dữ liệu sau khi augmenting
X_augmented = np.zeros((n_augmented_images, 128, 128, 3))
y_augmented = np.zeros((n_augmented_images, 3))

# Lấy dữ liệu sau khi augmenting và lưu vào mảng numpy
batch_index = 0
print(len(train_generator))
for x_batch, y_batch in train_generator:
    for i in range(x_batch.shape[0]):
        print(x_batch[i].dtype)
        X_augmented[batch_index] = x_batch[i]
        y_augmented[batch_index] = y_batch[i]
        batch_index += 1
    print(batch_index)
    if batch_index >= (n_augmented_images/1000):
        break

# Kết hợp dữ liệu ban đầu và dữ liệu được sinh ra từ augmenting
X_train_augmented = np.concatenate((X_train, X_augmented))
y_train_augmented = np.concatenate((y_train, y_augmented))

# Xác nhận số lượng ảnh đã được tăng lên
print("Tổng số ảnh được sinh ra trong quá trình augmenting:", n_augmented_images)
print("Tổng số ảnh sau khi làm giàu dữ liệu:", len(X_train_augmented))

vggmodel.fit_generator(aug_train.flow(X_train, y_train, batch_size= 64), epochs= 10, validation_data = aug_train.flow(X_test, y_test, batch_size=64), callbacks=callbacks_list)

In [ ]:

vggmodel.save("vggmodel.h5")

In [ ]:
import os
filesize = os.path.getsize('/content/weights-10-0.95.hdf5')
print(f'Kích thước của tập tin là {filesize} bytes.')
KB = 1024
MB = KB * 1024
GB = MB * 1024

filesize_in_kb = filesize / KB
filesize_in_mb = filesize / MB
filesize_in_gb = filesize / GB
print(filesize_in_gb)

Kích thước của tập tin là 663165272 bytes.
0.6176207885146141


In [ ]:
vggmodel.save("vggmodel.h5")

In [ ]:
from os import listdir
import cv2
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import random
from keras.models import  load_model
import sys


In [ ]:
my_model = get_model()
my_model.load_weights("/content/weights-10-0.95.hdf5")


In [ ]:
image_org = cv2.imread('/content/Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (1).jpg')

image_org = cv2.resize(image_org, dsize=None,fx=0.5,fy=0.5)
# Resize
image = image_org.copy()
image = cv2.resize(image, dsize=(128, 128))
image = image.astype('float')*1./255
# Convert to tensor
image = np.expand_dims(image, axis=0)
class_name = ['boot', 'sandal', 'shoe']
# Predict
predict = my_model.predict(image)
# print("This picture is: ", class_name[np.argmax(predict[0])])
print(np.max(predict[0]))
if (np.max(predict)>=0.8) :
# Show image
    
    print(class_name[np.argmax(predict[0])])

    

1/1 [==============================] - 1s 725ms/step
0.98557496
shoe
